In [80]:
# Env requirements
from dotenv import load_dotenv
import os
 
load_dotenv()

True

# OpenAI

In [81]:
from langchain_openai import OpenAI

open_api_key = os.getenv('OPENAI_API_KEY')

llm_openai = OpenAI(
    openai_api_key=open_api_key,
    temperature=0.6
)

user_input = "What's Israel Capital?"

print(llm_openai.invoke(user_input))



The capital of Israel is Jerusalem.


# HuggingFace

In [82]:
from langchain import HuggingFaceHub

hugging_face_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")

model_kwargs = {
    "temperature": 0,
    "max_length": 64
}

llm_hf = HuggingFaceHub(
    repo_id="google/flan-t5-large"
)

output = llm_hf.invoke("What is the capital of Russia? use first letter as capital")
output

'Moscow'

# Prompt Templates & LLMChain

In [83]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [85]:
country_input_variables = ['country']
country_prompt_query = "What is the capital of {country}?"

prompt_template = PromptTemplate(
    input_variables=country_input_variables,
    template=country_prompt_query
)

capital_chain = LLMChain(llm=llm_openai,prompt=prompt_template)
result = chain.invoke("Israel")
result['output']

"\n\n1. Visit the Old City: Explore the ancient walls of the Old City and wander through its four quarters - Jewish, Muslim, Christian, and Armenian. Visit iconic religious sites such as the Western Wall, Church of the Holy Sepulchre, and the Dome of the Rock.\n\n2. Take a tour of the Holocaust History Museum: Learn about the tragic history of the Holocaust at Yad Vashem, Israel's official memorial to the victims of the Holocaust.\n\n3. Walk through the Mahane Yehuda Market: Experience the vibrant atmosphere of this bustling market, filled with colorful stalls selling fresh produce, spices, and traditional Middle Eastern foods.\n\n4. Visit the Israel Museum: Explore the country's largest museum, which houses a vast collection of archaeological artifacts, Jewish art and culture, and contemporary art.\n\n5. Take a day trip to the Dead Sea: Relax and float in the mineral-rich waters of the Dead Sea, the lowest point on Earth. You can also visit nearby attractions such as Masada, an ancien

# Combining Multiple Chains Using Sequential Chain

### Creating Chain

In [86]:
capital_prompt = PromptTemplate(
    input_variables=country_input_variables,
    template=country_prompt_query
)

captial_chain = LLMChain(
    llm=llm_openai,
    prompt=capital_prompt
)

todo_location = ['capital']
todo_prompt_query = "Suggest me something to do in {capital}"

todo_template=PromptTemplate(
    input_variables=todo_location,
    template=todo_prompt_query
)

todo_chain = LLMChain(llm=llm_openai,prompt=todo_template)

### Executing Chain

In [87]:
from langchain.chains import SimpleSequentialChain

In [88]:
chain=SimpleSequentialChain(chains=[capital_chain,todo_chain])
result = chain.invoke("Israel")
result['output']

" There are many things you can do in Jerusalem, including:\n\n1. Visit the Old City: The Old City of Jerusalem is a UNESCO World Heritage Site and is home to many significant religious and historical sites, such as the Western Wall, the Church of the Holy Sepulchre, and the Dome of the Rock.\n\n2. Explore the Mahane Yehuda Market: This bustling outdoor market is a great place to experience the sights, sounds, and flavors of Jerusalem. You can find fresh produce, spices, and traditional Middle Eastern delicacies.\n\n3. Tour the Israel Museum: This world-renowned museum showcases a vast collection of artifacts and artworks that tell the story of Israel's history and culture.\n\n4. Walk the Ramparts: You can take a walk on the ancient walls of the Old City and get a unique perspective of the city and its surroundings.\n\n5. Visit Yad Vashem: This powerful museum and memorial is dedicated to the victims of the Holocaust and serves as a reminder of the importance of tolerance and remembran

# ChatModels with ChatOpenAI

In [118]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [123]:
chat_llm = ChatOpenAI(
    openai_api_key=open_api_key,
    temperature=0.5,
    model='gpt-3.5-turbo'
)

chat_llm(
    [
        SystemMessage(content="You are a comedian AI assistant"),
        HumanMessage(content="Tell a stupid joke that starts with 'from the river to the sea' and make it rhyme"),
    ]
)

AIMessage(content="I'm sorry, but I'm programmed to follow ethical guidelines, and I cannot provide or promote any offensive or inappropriate content. If you have any other requests or need assistance with something else, feel free to ask!")

# Prompt Template + LLM + Output Parsers

In [131]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [132]:
class CommaSeperatedOutput(BaseOutputParser):

    def parse(self,text:str):
        return text.strip().split(",")

In [134]:
template = "You are a helpful assistant. When the user gives any input you should generate 5 words in a comma-separated list"
human_template="{text}"

chatprompt = ChatPromptTemplate.from_messages([("system",template),("human",human_template)])

chain = chatprompt|chat_llm|CommaSeperatedOutput()

In [135]:
chain.invoke({"text":"smart"})

['intelligent', ' clever', ' sharp', ' quick-witted', ' astute']